In [11]:
import pandas as pd
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import accuracy_score, classification_report

# Load the datasets
train_df = pd.read_csv("/content/newtraint.tsv", sep="\t")
valid_df = pd.read_csv("/content/newvali.tsv", sep="\t")
test_df = pd.read_csv("/content/sentiment_labeled_dataset.tsv", sep="\t")

# Print initial column names
print("\n✅ Train Data Columns:", train_df.columns)

# Select only required columns: 'cs_query' (text) and 'Sentiment' (label)
train_df = train_df[['cs_query', 'Sentiment']]
valid_df = valid_df[['cs_query', 'Sentiment']]
test_df = test_df[['cs_query', 'Sentiment']]

# Strip whitespace from column names
train_df.columns = train_df.columns.str.strip()
valid_df.columns = valid_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()

# Print unique sentiment values before mapping
print("\n🔍 Unique Sentiments (Before Mapping):", train_df["Sentiment"].unique())

# Map Sentiment labels to numerical values
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
reverse_label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

train_df["Sentiment"] = train_df["Sentiment"].map(label_map)
valid_df["Sentiment"] = valid_df["Sentiment"].map(label_map)
test_df["Sentiment"] = test_df["Sentiment"].map(label_map)

# Drop rows with NaN values
train_df.dropna(inplace=True)
valid_df.dropna(inplace=True)
test_df.dropna(inplace=True)

# Convert Sentiment to integer type
train_df["Sentiment"] = train_df["Sentiment"].astype(int)
valid_df["Sentiment"] = valid_df["Sentiment"].astype(int)
test_df["Sentiment"] = test_df["Sentiment"].astype(int)

# Print unique sentiment values after mapping
print("\n✅ Unique Sentiments (After Mapping):", train_df["Sentiment"].unique())

# Verify dataset info
print("\n📊 Train Data Info:")
print(train_df.info())

# Print sample data
print("\n📌 Sample Train Data:")
print(train_df.head())

# Define improved model arguments for better training
model_args = ClassificationArgs()
model_args.num_train_epochs = 7  # Increased epochs for better generalization
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.learning_rate = 5e-6  # Lower LR for fine-tuning
model_args.weight_decay = 0.01  # Regularization to prevent overfitting
model_args.overwrite_output_dir = True
model_args.save_best_model = True
model_args.evaluate_during_training = True
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = True  # Save model after each epoch
model_args.output_dir = "xlm-roberta-sentiment-model"
model_args.best_model_dir = "xlm-roberta-best-model"
model_args.reprocess_input_data = True  # Ensure fresh processing each run
model_args.use_early_stopping = True  # Prevent overfitting
model_args.early_stopping_patience = 2  # Stop if no improvement

# Initialize the model
model = ClassificationModel(
    "xlmroberta", "xlm-roberta-base",
    num_labels=3,
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

print("\n✅ Model Loaded Successfully!")

# Train the model with logging enabled
print("\n🚀 Training Started...")
model.train_model(train_df, eval_df=valid_df, show_running_loss=True)

# Evaluate the model
print("\n📊 Evaluating Model...")
result, model_outputs, wrong_predictions = model.eval_model(valid_df, acc=accuracy_score)

print(f"\n✅ Model Accuracy: {result['acc']:.4f}")

# Function to predict sentiment with confidence scores
def predict_sentiment(text, model):
    predictions, raw_outputs = model.predict([text])
    confidence = torch.softmax(torch.tensor(raw_outputs[0]), dim=0).tolist()
    sentiment = reverse_label_map[predictions[0]]

    print("\n===============================")
    print(f"📝 **Text:** {text}")
    print(f"🤖 **Predicted Sentiment:** {sentiment}")
    print(f"🔹 **Confidence Scores:** {confidence}")
    print("===============================\n")
    return sentiment

# Predict on the test set
print("\n🚀 Running Predictions on Test Set...")
for text in test_df["cs_query"].tolist()[:10]:  # Run on first 10 test samples
    predict_sentiment(text, model)

# Save the trained model
model.save_model("my_simpletransformers_model")
print("\n✅ Model Saved Successfully!")



✅ Train Data Columns: Index(['en_query', 'cs_query', 'en_parse', 'cs_parse', 'domain', 'Sentiment'], dtype='object')

🔍 Unique Sentiments (Before Mapping): ['Neutral' 'Negative' 'Positive']

✅ Unique Sentiments (After Mapping): [1 0 2]

📊 Train Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2993 entries, 0 to 2992
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cs_query   2993 non-null   object
 1   Sentiment  2993 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 46.9+ KB
None

📌 Sample Train Data:
                                            cs_query  Sentiment
0  9 : 30 am ko Sunday Brunch ke liye ek naya wee...          1
1  danny ko message karo aur dekho ke he wants to...          1
2                  do ghante ke liye alarm set kardo          0
3  neil ke liye cake bake karne ke reminder ko mi...          2
4         Please mere chat requests ko retrieve kare          0


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:484: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(



✅ Model Loaded Successfully!

🚀 Training Started...


/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]


📝 **Text:** Waukegan ke Greenbay Road par sab se halka traffic kab hai
🤖 **Predicted Sentiment:** Negative
🔹 **Confidence Scores:** [0.8314967813198247, 0.14035496714602025, 0.02814825153415497]



0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


📝 **Text:** Me ab ek video message chodna chahta hu
🤖 **Predicted Sentiment:** Negative
🔹 **Confidence Scores:** [0.7177520268424857, 0.22240349640666293, 0.05984447675085121]


✅ Model Saved Successfully!


In [12]:
import pandas as pd
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import accuracy_score, classification_report

# Load the datasets
train_df = pd.read_csv("/content/newtraint.tsv", sep="\t")
valid_df = pd.read_csv("/content/newvali.tsv", sep="\t")
test_df = pd.read_csv("/content/sentiment_labeled_dataset.tsv", sep="\t")

# Print column names for debugging
print("\n✅ Train Data Columns:", train_df.columns)

# Select only required columns: 'cs_query' (input text) and 'Sentiment' (label)
train_df = train_df[['cs_query', 'Sentiment']]
valid_df = valid_df[['cs_query', 'Sentiment']]
test_df = test_df[['cs_query', 'Sentiment']]

# Strip whitespace from column names
train_df.columns = train_df.columns.str.strip()
valid_df.columns = valid_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()

# Print unique sentiment values before mapping
print("\n🔍 Unique Sentiments (Before Mapping):", train_df["Sentiment"].unique())

# Map Sentiment labels to numerical values
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
reverse_label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

train_df["Sentiment"] = train_df["Sentiment"].map(label_map)
valid_df["Sentiment"] = valid_df["Sentiment"].map(label_map)
test_df["Sentiment"] = test_df["Sentiment"].map(label_map)

# Drop rows with NaN values
train_df.dropna(inplace=True)
valid_df.dropna(inplace=True)
test_df.dropna(inplace=True)

# Convert Sentiment to integer type
train_df["Sentiment"] = train_df["Sentiment"].astype(int)
valid_df["Sentiment"] = valid_df["Sentiment"].astype(int)
test_df["Sentiment"] = test_df["Sentiment"].astype(int)

# Print unique sentiment values after mapping
print("\n✅ Unique Sentiments (After Mapping):", train_df["Sentiment"].unique())

# Verify dataset info
print("\n📊 Train Data Info:")
print(train_df.info())

# Print sample data
print("\n📌 Sample Train Data:")
print(train_df.head())

# Define improved model arguments for better training
model_args = ClassificationArgs()
model_args.num_train_epochs = 7  # Increased epochs for better generalization
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.learning_rate = 5e-6  # Lower LR for fine-tuning
model_args.weight_decay = 0.01  # Regularization to prevent overfitting
model_args.overwrite_output_dir = True
model_args.save_best_model = True
model_args.evaluate_during_training = True
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = True  # Save model after each epoch
model_args.output_dir = "muril-sentiment-model"
model_args.best_model_dir = "muril-best-model"
model_args.reprocess_input_data = True  # Ensure fresh processing each run
model_args.use_early_stopping = True  # Prevent overfitting
model_args.early_stopping_patience = 2  # Stop if no improvement

# Initialize the MURIL model
model = ClassificationModel(
    "bert", "google/muril-base-cased",
    num_labels=3,
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

print("\n✅ MURIL Model Loaded Successfully!")

# Train the model with logging enabled
print("\n🚀 Training Started...")
model.train_model(train_df, eval_df=valid_df, show_running_loss=True)

# Evaluate the model
print("\n📊 Evaluating Model...")
result, model_outputs, wrong_predictions = model.eval_model(valid_df, acc=accuracy_score)

print(f"\n✅ Model Accuracy: {result['acc']:.4f}")

# Function to predict sentiment with confidence scores
def predict_sentiment(text, model):
    predictions, raw_outputs = model.predict([text])
    confidence = torch.softmax(torch.tensor(raw_outputs[0]), dim=0).tolist()
    sentiment = reverse_label_map[predictions[0]]

    print("\n===============================")
    print(f"📝 **Text:** {text}")
    print(f"🤖 **Predicted Sentiment:** {sentiment}")
    print(f"🔹 **Confidence Scores:** {confidence}")
    print("===============================\n")
    return sentiment

# Save the trained model
model.save_model("muril_sentiment_model")
print("\n✅ Model Saved Successfully!")



✅ Train Data Columns: Index(['en_query', 'cs_query', 'en_parse', 'cs_parse', 'domain', 'Sentiment'], dtype='object')

🔍 Unique Sentiments (Before Mapping): ['Neutral' 'Negative' 'Positive']

✅ Unique Sentiments (After Mapping): [1 0 2]

📊 Train Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2993 entries, 0 to 2992
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cs_query   2993 non-null   object
 1   Sentiment  2993 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 46.9+ KB
None

📌 Sample Train Data:
                                            cs_query  Sentiment
0  9 : 30 am ko Sunday Brunch ke liye ek naya wee...          1
1  danny ko message karo aur dekho ke he wants to...          1
2                  do ghante ke liye alarm set kardo          0
3  neil ke liye cake bake karne ke reminder ko mi...          2
4         Please mere chat requests ko retrieve kare          0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ MURIL Model Loaded Successfully!

🚀 Training Started...


/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]


📝 **Text:** Waukegan ke Greenbay Road par sab se halka traffic kab hai
🤖 **Predicted Sentiment:** Negative
🔹 **Confidence Scores:** [0.5791509331621566, 0.2275197279200937, 0.19332933891774973]



0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


📝 **Text:** Me ab ek video message chodna chahta hu
🤖 **Predicted Sentiment:** Negative
🔹 **Confidence Scores:** [0.3765548450419363, 0.317158716150268, 0.30628643880779555]


✅ Model Saved Successfully!


In [13]:
import torch
from simpletransformers.classification import ClassificationModel

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Define paths to fine-tuned models
xlmr_model_path = "xlm-roberta-best-model"  # Change if needed
muril_model_path = "muril-best-model"  # Change if needed

# Load the trained XLM-RoBERTa model
xlmr_model = ClassificationModel(
    "xlmroberta", xlmr_model_path,
    use_cuda=torch.cuda.is_available()
)

# Load the trained MURIL model
muril_model = ClassificationModel(
    "bert", muril_model_path,  # MURIL is based on BERT
    use_cuda=torch.cuda.is_available()
)

print("\n✅ Both Models Loaded Successfully!")

# Define label mapping
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
reverse_label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

# Function to predict sentiment
def predict_sentiment(text, model, model_name):
    predictions, raw_outputs = model.predict([text])
    return reverse_label_map[predictions[0]]

# Example Sentences with Ground Truth Sentiment
sample_texts = [
    {"text": "I love this product! It's amazing!", "ground_truth": "Positive"},
    {"text": "The service was okay, nothing special.", "ground_truth": "Neutral"},
    {"text": "This is the worst experience I've ever had.", "ground_truth": "Negative"}
]

# Make predictions with both models
for sample in sample_texts:
    text = sample["text"]
    ground_truth = sample["ground_truth"]

    xlmr_sentiment = predict_sentiment(text, xlmr_model, "XLM-RoBERTa")
    muril_sentiment = predict_sentiment(text, muril_model, "MURIL")

    print("\n===============================")
    print(f"📝 **Text:** {text}")
    print(f"✅ **Ground Truth:** {ground_truth}")
    print(f"🤖 **XLM-RoBERTa Prediction:** {xlmr_sentiment}")
    print(f"🤖 **MURIL Prediction:** {muril_sentiment}")
    print("===============================\n")


Using device: cuda


/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:484: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(



✅ Both Models Loaded Successfully!


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


📝 **Text:** This is the worst experience I've ever had.
✅ **Ground Truth:** Negative
🤖 **XLM-RoBERTa Prediction:** Negative
🤖 **MURIL Prediction:** Negative

